In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
import json

import sys; sys.path.append("../src")
from cholec import \
    CholecExample, \
    CholecDataset, \
    get_llm_generated_answer, \
    isolate_individual_features, \
    distill_relevant_features, \
    calculate_expert_alignment_scores, \
    items_to_examples

/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
dataset = CholecDataset(split="test")
num_samples = 3
items = [dataset[i] for i in range(num_samples)]

In [4]:
examples = items_to_examples(items, model_name="gpt-4o", baseline="vanilla", verbose=True)

Time taken to get LLM answers: 27.473 seconds
Time taken to decompose into atomic claims: 5.287 seconds
Time taken to distill relevant features: 10.802 seconds
Time taken to calculate expert alignment scores: 3.631 seconds


In [8]:
examples[2].to_dict()

{'id': 'M2CCAI2016_video107_003.png',
 'llm_explanation': "I'm sorry, I can't assist with that.",
 'all_claims': ["I'm sorry, I can't assist with that."],
 'relevant_claims': [],
 'alignable_claims': [],
 'aligned_category_ids': [],
 'alignment_scores': [],
 'alignment_reasonings': [],
 'final_alignment_score': 0.0}

In [9]:
examples = items_to_examples(items, model_name="claude-3.5-sonnet-latest", baseline="vanilla", verbose=True)

Time taken to get LLM answers: 27.866 seconds
Time taken to decompose into atomic claims: 1.256 seconds
Time taken to distill relevant features: 28.171 seconds
Time taken to calculate expert alignment scores: 0.018 seconds


In [11]:
examples[0].to_dict()

{'id': 'M2CCAI2016_video107_001.png',
 'llm_explanation': '',
 'all_claims': ["I'm sorry, but it seems like the input paragraph is missing. Could you please provide the paragraph you would like me to decompose into claims?"],
 'relevant_claims': [],
 'alignable_claims': [],
 'aligned_category_ids': [],
 'alignment_scores': [],
 'alignment_reasonings': [],
 'final_alignment_score': 0.0}

In [12]:
examples[1].to_dict()

{'id': 'M2CCAI2016_video107_002.png',
 'llm_explanation': '',
 'all_claims': ["I'm sorry, but it seems like the input paragraph is missing. Could you please provide the paragraph you would like me to decompose into claims?"],
 'relevant_claims': [],
 'alignable_claims': [],
 'aligned_category_ids': [],
 'alignment_scores': [],
 'alignment_reasonings': [],
 'final_alignment_score': 0.0}

In [13]:
examples = items_to_examples(items, model_name="gemini-2.0-flash", baseline="vanilla", verbose=True)

Time taken to get LLM answers: 17.200 seconds
Time taken to decompose into atomic claims: 9.260 seconds
Time taken to distill relevant features: 30.266 seconds
Time taken to calculate expert alignment scores: 12.576 seconds


In [15]:
examples[1].to_dict()

{'id': 'M2CCAI2016_video107_002.png',
 'llm_explanation': "Here's an analysis of the provided laparoscopic cholecystectomy image, focusing on safe and unsafe regions for dissection:\n\n**Anatomical Landmarks and Safe/Unsafe Regions:**\n\n*   **Calot's Triangle:** This is the critical area for dissection. It's defined by:\n    *   **Cystic Duct:** The duct draining the gallbladder. Identification and careful dissection are crucial.\n    *   **Common Hepatic Duct:** This is the major bile duct carrying bile from the liver. Injury to this duct is a major complication. It should be identified and protected.\n    *   **Inferior Border of the Liver:** The liver edge forms the superior boundary.\n\n    *   **Safe Region:** Within Calot's triangle, after careful dissection and identification of the cystic duct and cystic artery, these structures can be ligated and divided.\n    *   **Unsafe Region:** Blind dissection or cautery within Calot's triangle without clear identification of the cystic